In [1]:
%matplotlib inline

import numpy as np

from matplotlib import rc
import matplotlib.pyplot as plt
import scipy.stats as st

import sys
sys.path.append("/home/joshua/Documents/Uni/Year4/dissertation/catkin_ws/src/linefollow_gazebo/scripts")


import helper as h
from CameraModel import Camera
from RoadModel import Road, NormalProbabilityDist
from Path import Path

In [2]:
def get_predicted_error_radius(camera_height,
                               ground_distance, 
                               pixel_area,
                               sigma_pos=0.03/2.0,
                               sigma_orient_deg=0.5/2,
                               alg_error=4.0, # note: changed
                               verbose=False,
                               orientation_percent=0.9,
                               pos_percent=0.9,
                               alg_percent=0.9,
                               use_err_xy=True,
                               use_err_z=True,
                               use_err_orient=True,
                               use_err_pixel=True
                              ):
    # TODO this is wrong implementation! Doesn't matter though really as long as use is consistent
    # but really it should be -1*norm.ppf((1-percent)/2.0)
    # as it is is an underestimate

    stddevs_orientation = st.norm.ppf(orientation_percent)
    stddevs_position = st.norm.ppf(pos_percent)
    stddevs_alg = st.norm.ppf(alg_percent)
    
    
    # NOT invariant between pixels
    z_error = (ground_distance/camera_height)*stddevs_position*sigma_pos if use_err_z else 0
    
    # NOT invariant between pixels
    if use_err_orient:
        t = stddevs_orientation*np.deg2rad(sigma_orient_deg)
        alpha = np.arctan2(ground_distance, camera_height)
        beta = alpha + t
        if beta > np.pi/2: # causes tan to roll over into negatives...
            orientation_error = 9999999999
        else:
            orientation_error = np.tan(beta)*camera_height - ground_distance
    else:
        orientation_error = 0
    
    pixel_error = np.sqrt(2*pixel_area) if use_err_pixel else 0
    
#     print("Z error: {0}".format(z_error))
#     print("orien error: {0}".format(orientation_error))
#     print("pixel_error: {0}".format(pixel_error))
    total = z_error + orientation_error + (alg_error+1)/2.0 * stddevs_alg * pixel_error
    return total, z_error, orientation_error, alg_error*pixel_error

In [3]:
def error_radius_across_pixels(camera):
    
    image = np.zeros(shape=(camera.R_y, camera.R_x+1)) # add one because using pixel as bottom left corner! Symmetry matters here
    camera_x, camera_y, camera_z = camera.position
    camera_ground_pos = np.array([camera_x, camera_y, 0.0])

    for (y,x), _ in np.ndenumerate(image):

        ground_point = camera.pixel_to_plane(x, y)
        pixel_area = camera.plane_area_of_pixel(x,y)

        if ground_point is None or pixel_area is None:
            image[y, x] = 999999999999
            continue

        ground_distance = np.linalg.norm(ground_point - camera_ground_pos)

        image[y, x] = get_predicted_error_radius(camera_z, ground_distance, pixel_area)[0]

        return image
        

In [4]:
# basically follows the pseudocode from wikipedia for continuous hillclimbing

def hillclimb(score_function, parameter_ranges, parameter_initializations=None, finish_epsilon=0.1, parameter_step_fraction=0.05, acceleration=1.2):
    """ Takes a scoring function, a set of parameter ranges, parameter initializations if desired, the finishing epsilon, and the fraction of the parameter range that one step takes initially    
    """
    
    params = []
    # if no initializations, choose one randomly
    if parameter_initializations is None:
        for param_range in parameter_ranges:
            params.append(np.random.uniform(*param_range))
    else:
        params = parameter_initializations
    params = np.array(params)
    
    params_ranges = np.array([upper - lower for (lower, upper) in parameter_ranges])
    params_step = params_ranges * parameter_step_fraction
    
    movements = np.array([-acceleration, -1.0/acceleration, 0, 1.0/acceleration, acceleration])
    
    
    score = score_function(params)
    while True:
        
        for i, param in enumerate(params):
            
            best_move = -1
            best_score = -9999999
            
            # do 5 steps per param
            # this is relatively inefficient with an expensive cost function
            # since we evaluate '0' parameter each time, which doesn't need to be evaluated...
            for j, move in enumerate(movements):
                params[i] += params_step[i] * move
                
                # check if move is out of bounds!
                if params[i] < parameter_ranges[i][0] or params[i] > parameter_ranges[i][1]:
                    # then skip, can always revert to '0' or no change
                    params[i] -= params_step[i] * move
                    continue
                
                new_score = score_function(params)
                params[i] -= params_step[i] * move
                
                if new_score > best_score:
                    best_score = new_score
                    best_move = j
            
            if movements[best_move] == 0:
                params_step[i] /= acceleration
            else:
                params[i] += params_step[i] * movements[best_move]
                params_step[i] = params_step[i] * movements[best_move] # accelerate step size
           
        updated_score = score_function(params)
        if updated_score - score < finish_epsilon:
            return (score, params)
        score = updated_score
            


In [5]:
# Hill climbing with random restarts

def random_restart_hillclimb(score_function, parameter_ranges, finish_epsilon=0.1, parameter_step_fraction=0.01, acceleration=1.2, num_restarts=100, initialization_generator=None, append_to=None):
    
    if append_to is None:
        local_score_maxima = []
        local_maxima_params = []
    else:
        local_score_maxima = append_to[0]
        local_maxima_params = append_to[1]
    
    for _ in range(num_restarts):
        if initialization_generator is not None:
            init_params = initialization_generator(parameter_ranges)
        else:
            init_params = None
        
        try:
            score, params = hillclimb(score_function, parameter_ranges, parameter_initializations=init_params, finish_epsilon=finish_epsilon, parameter_step_fraction=parameter_step_fraction, acceleration=acceleration)
            print("{0}, params: {1}".format(score, params))

        except Exception as e: # something...
            print("error!", e)
        
        local_score_maxima.append(score)
        local_maxima_params.append(params)
    return (local_score_maxima, local_maxima_params)
        

In [6]:
road_prob_dist = NormalProbabilityDist(two_stddev=0.4)

camera_height = 6.0
resolution = (100, 100)
yaw_range = 90 # degrees +- from center yaw position

curvature = 0.0
length = 40.0 # does matter! can see end of road!

path = Path()
path.add_segment(curvature, length)
road = Road(path=path, probability_distribution=road_prob_dist, width=3.0, side_offset=1.5)

# place camera halfway along road
allowed_positions_orientations = road.get_allowed_camera_positions_at_distance(length/2.0)
lhs_pos, lhs_direction = allowed_positions_orientations[0]
rhs_pos, rhs_direction = allowed_positions_orientations[1]

# just stick with lhs for now, symmetric with straight road

camera_position = lhs_pos + np.array([0.0, 0.0, camera_height])
camera_pitch_range = np.array([31.0, 85.0])

# convert lhs_direction into a yaw facing the road perpendicularly
center_yaw_rad = np.arccos(np.dot(np.array([1.0,0,0]), h.normalize(lhs_direction)))
center_yaw_deg = np.rad2deg(center_yaw_rad)
camera_yaw_range = np.array([center_yaw_deg - yaw_range, center_yaw_deg + yaw_range])

fov_h_range = np.array([20.0, 120.0])
fov_v_range = fov_h_range.copy()


In [7]:
print(camera_position)
print(center_yaw_deg)


[ 20.  -3.   6.]
90.0


In [8]:
def score_function(params):
    pitch, yaw, h_fov, v_fov = params
    c = Camera(camera_position, pitch, yaw)
    c.set_resolution(*resolution)
    c.set_fov(h_fov, v_fov)
    
    
    error_bounds = error_radius_across_pixels(c)
    pixel_probabilities = c.get_pixel_probabilities_for_road(road)
    
    target = (1/(1+(error_bounds/3.0)**2)) * pixel_probabilities
    score = np.sum(target)
    print("--score, params: {0}, {1}".format(score, params))
    return score
    

In [9]:
scores_params = [[],[]]

In [10]:
param_ranges = np.array([
    camera_pitch_range, camera_yaw_range, fov_h_range, fov_v_range
])


random_restart_hillclimb(score_function, param_ranges, finish_epsilon=0.1, parameter_step_fraction=0.1, acceleration=1.5, num_restarts=100, append_to=scores_params)

--score, params: 1.11806536012, [  47.70754652  130.93235042  118.45720813   22.88764973]
--score, params: 0.964181101583, [  39.60754652  130.93235042  118.45720813   22.88764973]
--score, params: 1.05225359554, [  44.10754652  130.93235042  118.45720813   22.88764973]
--score, params: 1.11806536012, [  47.70754652  130.93235042  118.45720813   22.88764973]
--score, params: 1.17945839844, [  51.30754652  130.93235042  118.45720813   22.88764973]
--score, params: 1.24964221087, [  55.80754652  130.93235042  118.45720813   22.88764973]
--score, params: 3.40002077021, [  55.80754652  103.93235042  118.45720813   22.88764973]
--score, params: 1.69235433492, [  55.80754652  118.93235042  118.45720813   22.88764973]
--score, params: 1.24964221087, [  55.80754652  130.93235042  118.45720813   22.88764973]
--score, params: 1.02606340934, [  55.80754652  142.93235042  118.45720813   22.88764973]
--score, params: 0.883449683828, [  55.80754652  157.93235042  118.45720813   22.88764973]
--score,

--score, params: 3.09067977732, [  70.65547409  109.68270669   79.29855437   49.33556693]
--score, params: 3.02646990498, [  62.55547409  109.68270669   79.29855437   49.33556693]
--score, params: 3.04908514814, [  67.05547409  109.68270669   79.29855437   49.33556693]
--score, params: 3.09067977732, [  70.65547409  109.68270669   79.29855437   49.33556693]
--score, params: 2.98499798426, [  74.25547409  109.68270669   79.29855437   49.33556693]
--score, params: 2.60439846559, [  78.75547409  109.68270669   79.29855437   49.33556693]
--score, params: 2.51638569422, [  70.65547409  136.68270669   79.29855437   49.33556693]
--score, params: 2.69751863311, [  70.65547409  121.68270669   79.29855437   49.33556693]
--score, params: 3.09067977732, [  70.65547409  109.68270669   79.29855437   49.33556693]
--score, params: 2.9870779096, [ 70.65547409  97.68270669  79.29855437  49.33556693]
--score, params: 2.98532342268, [ 70.65547409  82.68270669  79.29855437  49.33556693]
--score, params: 2.

--score, params: 2.60368816555, [ 48.46937087  36.98870548  61.8987449   43.78342311]
--score, params: 2.60368816555, [ 48.46937087  36.98870548  61.8987449   43.78342311]
--score, params: 1.95925384747, [ 36.31937087  36.98870548  61.8987449   43.78342311]
--score, params: 2.37686682492, [ 43.06937087  36.98870548  61.8987449   43.78342311]
--score, params: 2.60368816555, [ 48.46937087  36.98870548  61.8987449   43.78342311]
--score, params: 2.80738447104, [ 53.86937087  36.98870548  61.8987449   43.78342311]
--score, params: 3.00534557928, [ 60.61937087  36.98870548  61.8987449   43.78342311]
--score, params: 2.55596733235, [ 60.61937087  18.98870548  61.8987449   43.78342311]
--score, params: 3.00534557928, [ 60.61937087  36.98870548  61.8987449   43.78342311]
--score, params: 3.68143993697, [ 60.61937087  54.98870548  61.8987449   43.78342311]
--score, params: 3.39335491615, [ 60.61937087  77.48870548  61.8987449   43.78342311]
--score, params: 2.7867592591, [ 60.61937087  54.98870

--score, params: 1.13467583851, [  31.62003324   61.78342327   41.91747094  117.78306129]
--score, params: 1.95455395912, [ 31.62003324  61.78342327  41.91747094  87.78306129]
--score, params: 1.63305447977, [ 43.77003324  61.78342327  41.91747094  87.78306129]
--score, params: 1.78084808096, [ 37.02003324  61.78342327  41.91747094  87.78306129]
--score, params: 1.95455395912, [ 31.62003324  61.78342327  41.91747094  87.78306129]
--score, params: 1.93918503098, [ 31.62003324  79.78342327  41.91747094  87.78306129]
--score, params: 1.9472354659, [ 31.62003324  69.78342327  41.91747094  87.78306129]
--score, params: 1.95455395912, [ 31.62003324  61.78342327  41.91747094  87.78306129]
--score, params: 1.95835048495, [ 31.62003324  53.78342327  41.91747094  87.78306129]
--score, params: 1.94690173198, [ 31.62003324  43.78342327  41.91747094  87.78306129]
--score, params: 1.95835553297, [ 31.62003324  53.78342327  31.91747094  87.78306129]
--score, params: 1.9583555703, [ 31.62003324  53.78

--score, params: 3.69620169749, [ 35.62003324  36.89453438  20.80635983  44.44972795]
--score, params: 3.94301575391, [ 37.22003324  36.89453438  20.80635983  44.44972795]
--score, params: 4.10490347678, [ 39.22003324  36.89453438  20.80635983  44.44972795]
--score, params: 3.89117410539, [ 39.22003324  42.22786771  20.80635983  44.44972795]
--score, params: 4.028755524, [ 39.22003324  39.26490475  20.80635983  44.44972795]
--score, params: 4.10490347678, [ 39.22003324  36.89453438  20.80635983  44.44972795]
--score, params: 4.14906312107, [ 39.22003324  34.52416401  20.80635983  44.44972795]
--score, params: 4.16294988136, [ 39.22003324  31.56120104  20.80635983  44.44972795]
--score, params: 3.98031721203, [ 39.22003324  31.56120104  27.47302649  44.44972795]
--score, params: 4.11883175159, [ 39.22003324  31.56120104  23.76932279  44.44972795]
--score, params: 4.16294988136, [ 39.22003324  31.56120104  20.80635983  44.44972795]
--score, params: 2.9903545405, [ 39.22003324  31.5612010

...

--score, params: 4.90270247924, [ 73.58526129  86.03285397  20.73330433  22.13154177]
--score, params: 6.59898144649, [ 71.98526129  86.03285397  20.73330433  22.13154177]
--score, params: 6.9439058812, [ 70.38526129  86.03285397  20.73330433  22.13154177]
--score, params: 6.92319938879, [ 68.38526129  86.03285397  20.73330433  22.13154177]
--score, params: 6.94191353482, [ 70.38526129  91.3661873   20.73330433  22.13154177]
--score, params: 6.94204857078, [ 70.38526129  88.40322434  20.73330433  22.13154177]
--score, params: 6.9439058812, [ 70.38526129  86.03285397  20.73330433  22.13154177]
--score, params: 6.94330284835, [ 70.38526129  83.6624836   20.73330433  22.13154177]
--score, params: 6.92527703068, [ 70.38526129  80.69952064  20.73330433  22.13154177]
--score, params: 6.92886668219, [ 70.38526129  86.03285397  35.73330433  22.13154177]
--score, params: 6.93861342867, [ 70.38526129  86.03285397  27.399971    22.13154177]
--score, params: 6.9439058812, [ 70.38526129  86.0328539

--score, params: 2.24410349347, [ 42.11607667  53.42245143  61.04725441  71.79831059]
--score, params: 2.24751863638, [ 42.11607667  45.42245143  61.04725441  71.79831059]
--score, params: 2.19516286975, [ 42.11607667  37.42245143  61.04725441  71.79831059]
--score, params: 1.99640468183, [ 42.11607667  27.42245143  61.04725441  71.79831059]
--score, params: 2.02448730094, [ 42.11607667  45.42245143  71.04725441  71.79831059]
--score, params: 2.16632240994, [ 42.11607667  45.42245143  65.49169886  71.79831059]
--score, params: 2.24751863638, [ 42.11607667  45.42245143  61.04725441  71.79831059]
--score, params: 2.28497315515, [ 42.11607667  45.42245143  56.60280997  71.79831059]
--score, params: 2.29594623542, [ 42.11607667  45.42245143  51.04725441  71.79831059]
--score, params: 1.54240351434, [ 42.11607667  45.42245143  51.04725441  94.29831059]
--score, params: 1.91950980346, [ 42.11607667  45.42245143  51.04725441  81.79831059]
--score, params: 2.29594623542, [ 42.11607667  45.4224

--score, params: 7.35806915196, [ 55.41607667  44.03973538  21.60280997  24.29831059]
--score, params: 7.33056952848, [ 55.41607667  45.09323332  21.60280997  24.29831059]
--score, params: 7.27814623604, [ 55.41607667  46.41010575  21.60280997  24.29831059]
--score, params: 6.9304232966, [ 55.41607667  44.03973538  28.26947663  24.29831059]
--score, params: 7.2483691996, [ 55.41607667  44.03973538  24.56577293  24.29831059]
--score, params: 7.35806915196, [ 55.41607667  44.03973538  21.60280997  24.29831059]
--score, params: 5.74910275799, [ 55.41607667  44.03973538  21.60280997  30.96497726]
--score, params: 6.56417655802, [ 55.41607667  44.03973538  21.60280997  27.26127356]
--score, params: 7.35806915196, [ 55.41607667  44.03973538  21.60280997  24.29831059]
--score, params: 8.2094528394, [ 55.41607667  44.03973538  21.60280997  21.33534763]
--score, params: 8.2094528394, [ 55.41607667  44.03973538  21.60280997  21.33534763]
--score, params: 8.10503998014, [ 53.01607667  44.03973538

--score, params: 4.04188519386, [ 49.95065781  84.20730286  33.00998841  38.78130536]
--score, params: 4.06516088321, [  49.95065781  102.20730286   33.00998841   38.78130536]
--score, params: 4.19454302072, [  49.95065781  120.20730286   33.00998841   38.78130536]
--score, params: 4.65238995125, [  49.95065781  142.70730286   33.00998841   38.78130536]
--score, params: 3.04429379573, [  49.95065781  142.70730286   55.50998841   38.78130536]
--score, params: 4.05299560072, [  49.95065781  142.70730286   43.00998841   38.78130536]
--score, params: 4.65238995125, [  49.95065781  142.70730286   33.00998841   38.78130536]
--score, params: 4.6876968403, [  49.95065781  142.70730286   23.00998841   38.78130536]
--score, params: 3.63892012192, [  49.95065781  142.70730286   23.00998841   48.78130536]
--score, params: 4.16461612674, [  49.95065781  142.70730286   23.00998841   43.2257498 ]
--score, params: 4.6876968403, [  49.95065781  142.70730286   23.00998841   38.78130536]
--score, params:

KeyboardInterrupt: 

In [11]:
print(len(scores_params))
print(len(scores_params[0]), len(scores_params[1]))
hillclimb_results_scores = np.array(scores_params[0])
hillclimb_results_params = np.array(scores_params[1])
sorted_indices = np.argsort(hillclimb_results_scores)

2
(39, 39)


In [12]:
for index in sorted_indices:
    print("{0:.4f}\t ---\t{1}".format(hillclimb_results_scores[index], hillclimb_results_params[index]))


2.0708	 ---	[  76.86853428   23.21032921   26.27815559  102.60383155]
2.3175	 ---	[ 84.86166469  17.62045709  72.58595691  30.77742854]
2.4080	 ---	[  82.96805716  170.84599033   68.63206853   44.35070675]
2.4179	 ---	[  31.66146978  121.27515749   22.46309088   75.75621945]
2.5633	 ---	[ 32.69434766  88.85269004  23.73223633  65.4720451 ]
2.5977	 ---	[ 31.86581291  54.76693509  21.20962637  71.60767824]
2.6824	 ---	[ 83.35813221  14.26971378  61.89564327  32.23925538]
2.6936	 ---	[ 79.95176734   1.59891782  62.19665906  26.22593004]
2.8274	 ---	[  82.68513916  177.53872782   60.12092165   34.21057176]
3.0710	 ---	[ 39.31269545  87.66388745  25.01140436  53.56150822]
5.1884	 ---	[  46.04850165  146.50378408   26.80615751   34.99621263]
5.7361	 ---	[ 40.80504676  27.31963219  22.5532839   29.70477397]
6.1735	 ---	[  60.29140962  129.04450171   27.19235409   27.27412558]
6.4399	 ---	[  44.94634605  153.11754645   21.96011242   23.58640057]
6.5873	 ---	[  65.58880743  118.57159558   23.04

In [13]:
np.save("data/hillclimbing/V2_0_curve_pitch_yaw_fov_6m_scores_moredata.npy", hillclimb_results_scores[sorted_indices])
np.save("data/hillclimbing/v2_0_curve_pitch_yaw_fov_6m_params_moredata.npy", hillclimb_results_params[sorted_indices])


## Try Different Curvatures

In [ ]:
curvatures = [0.001, 0.005, 0.01, 0.05, 0.1, 0.2, 0.3]
# curvatures = [0.1]
hillclimb_scores_params = []
# separate cell => abort and can save partial results

In [ ]:
for curvature in curvatures:
    print("---- Curvature: {0} ----".format(curvature))

    camera_height = 6.0
#     resolution = (100, 100) # use same resolution for all
    fov_deg = (60.0, 60.0)
    yaw_range = 90 # degrees +- from center yaw position

    length = 2*np.pi/curvature # 1 full revolution

    path = Path()
    path.add_segment(curvature, length)
    road = Road(path=path, probability_distribution=road_prob_dist, width=3.0, side_offset=1.5)

    # place camera halfway along road
    allowed_positions_orientations = road.get_allowed_camera_positions_at_distance(length/2.0)
    lhs_pos, lhs_direction = allowed_positions_orientations[0]
    rhs_pos, rhs_direction = allowed_positions_orientations[1]

    #TODO figure out which one is LHS versus RHS for positive curvature...
    # ====> the first element (lhs here) is OUT ie. away from center of curvature
    # ====> LHS is good for first tests of outside
    
    camera_position = lhs_pos + np.array([0.0, 0.0, camera_height])
    camera_pitches = np.arange(31.0, 92.0, 3)

    # convert lhs_direction into a yaw facing the road perpendicularly
    center_yaw_rad = np.arccos(np.dot(np.array([1.0,0,0]), h.normalize(lhs_direction)))
    center_yaw_deg = np.rad2deg(center_yaw_rad) + 180 # TODO figure out automatically if the +180 is required
    camera_yaws = np.arange(center_yaw_deg - yaw_range, center_yaw_deg + yaw_range + 10, 10.0)
    
    # cylinder charactistics
    road_pt = road.path.get_point_at(length/2.0)
    road_normal = road.path.get_normal_at(length/2.0)
    road_radius = np.abs(1.0/road.path.get_curvature_at(length/2.0))
    cylinder_radius = np.abs(1.0/road.path.get_curvature_at(length/2.0)) 
    cylinder_radius -= road.halfwidth 
    cylinder_radius -= road.side_offset
    cylinder_center = road_pt + road_normal * road_radius
    cylinder_direction = np.array([0.0, 0.0, 1.0])
    
    x_unit, y_unit = np.array([1.0,0,0]), np.array([0.0, 1.0, 0.0])
    
    path_points = []
    for d in np.arange(0, length, 0.5):
        path_points.append(road.path.get_point_at(d))
    path_points = np.array(path_points)
    
    
    print("Cylinder radius: {0}".format(cylinder_radius))
    print("Road radius: {0}".format(1/road.path.get_curvature_at(length/2.0)))

    
    def score_function(params):
        pitch, yaw, h_fov, v_fov = params
        c = Camera(camera_position, pitch, yaw)
        c.set_resolution(*resolution)
        c.set_fov(h_fov, v_fov)

        error_bounds = error_radius_across_pixels(c)
        pixel_probabilities = c.get_pixel_probabilities_for_road(road)

        target = (1/(1+(error_bounds/3.0)**2)) * pixel_probabilities
        score = np.sum(target)
        print("--score, params: {0}, {1}".format(score, params))
        return score

    param_ranges = np.array([
        camera_pitch_range, camera_yaw_range, fov_h_range, fov_v_range
    ])
    
    hillclimb_scores_params.append([[],[]]) # append a new list for scores and corresponding params
    random_restart_hillclimb(score_function, param_ranges, finish_epsilon=0.1, parameter_step_fraction=0.1, acceleration=1.5, num_restarts=100, append_to=hillclimb_scores_params[-1])

In [ ]:
for i, (scores, params) in enumerate(hillclimb_scores_params):
    scores, params = np.array(scores), np.array(params)
    c = curvatures[i]
    sorted_indices = np.argsort(scores)
    print("---Curvature Top Scores: {0}---".format(c))
    for index in sorted_indices:
        print("Score: {0},   params: {1}".format(scores[index], params[index]))
    r = int(1.0/curvature)
    np.save("data/hillclimbing/V2_{0}_radius__pitch_yaw_fov_6m_scores.npy".format(r), scores[sorted_indices])
    np.save("data/hillclimbing/v2_{0}_radius_pitch_yaw_fov_6m_params.npy".format(r), params[sorted_indices])